In [ ]:
#pour effectuer correctement le scrapping nous aurons besoin de ces bibiliotéques : 

import re  #pour analyser les chaines de caractéres
import requests, bs4 #requests pour interagir avec le Web #bs4 pour BeautifulSoup
from requests import get #Pour atteindre envoyer une requete htpp 
from bs4 import BeautifulSoup #Pour scrapper le site avec les balises html
from IPython.core.display import clear_output  #Pour nettoyer le contenu de la trame (requete http)
from warnings import warn #controler les alertes
from time import sleep     #pour suspendre le temps d'execution durant un temps donné
from random import randint #pour renvoyer aléatoirement un entier
from time import time      #pour affiner le controle des requetes avec le temps (en secondes)
  from IPython.core.display import clear_output  #Pour nettoyer le contenu de la trame (requete http)
#pour scrapper le site il est important qu'il identifie via la requete HTTP que ne nous sommes pas un robot
#car cela peut bloquer l'acces au site via une requete http get

#on rentre les information de notre machine qui seront dans la trame http
headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko)Chrome/70.0.3538.77 Safari/537.36'}

#nous allons parcourir 196 pages, mais ici on initailise à bcp plus car on eu un pb avec l'iteration de la boucle
#et le nombre de pages donc on a décidé d'initialisé à plus en mettant un nombre assez large
pages = [str(i) for i in range(0,10000,1)]
#Listes des informations à prendre dans les pages nom,date de sortie, plateforme,metascore,userscore
names = []
release_dates = []
platform= []
meta_scores =[]
user_scores = []
#paramétrage de la boucle du monitoring afin de visualiser l'enchainement des requetes de la boucle
#plus de 196 pages et 56000 jeux à prendre
start_time = time()
requests = 0

#iteration de la boucle avec la liste page (0=>10000)
for page in pages:
    

    #avec la fonction get on créer la requete http
    game = get('https://www.metacritic.com/browse/games/score/metascore/all/all/filtered?page='+ page,headers = headers)
    #pause dans la boucle de 8 à 20 secondes
    sleep(randint(8,20))
    #initialisition de la variable requetes et du temps d'execution pour l'affichage du monitoring
    requests += 1
    elapsed_time = time() - start_time
    #affichage du monitoring
    print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
    clear_output(wait = True)
    #affihce un message d'alerte si au moins 200 requetes ne sont pas executer
    if game.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests,response.status_code))
    #arrete la boucle à partir de plus de 196 requetes
    if requests > 196:
        warn('Number of requests was greater than expected.')
        break
    #analyser le contenu de la réponse de la requete http dans la variable game_soup 
    game_soup = BeautifulSoup(game.text, 'html.parser')
    #trouve le tag majeur pour nos infos à recupérer
    container = game_soup.find_all('td', class_ = 'clamp-summary-wrap')
    #itération dans le majeur tag trouver
    for con in container:
        #scrape le nom du jeux
        name = con.find('h3').text
        names.append(name)
        #scrape la date de sortie
        release_date = con.select('div.clamp-details span')[2].text
        release_dates.append(release_date)
        #scrape la plateforme
        plateforme = con.select('div.clamp-details span')[1].text.strip()
        platform.append(plateforme)
        #scrape metascore
        meta_score= con.select('a.metascore_anchor div')[0].text
        meta_scores.append(meta_score)
        #scrape user score
        user_score = con.select('a.metascore_anchor div')[2].text
        user_scores.append(user_score)
        
#sotck dans le df movie_df toutes les infos        
import pandas as pd
movie_df = pd.DataFrame({'Movie_names': names,
'Release_dates': release_dates,
'Platform': platform,
'Meta_scores': meta_scores,
'User_scores': user_scores})
print(movie_df.info())


#rename du nouveau dataframe pour hamrmoniser 
game_df = movie_df.rename(columns={"Movie_names": "Name", "Release_dates": "Year"})

#convertion en format date 
game_df['Year'] = pd.to_datetime(game_df['Year'])
#extraction de l'année avec la fonction year
game_df['year'] = game_df['Year'].dt.year
#extraction du df en un csv
game_df.to_csv('metagames.csv')
Jointure entre l'ancien Dataframe VGSALES et le nouveau DF avec les scores METAGAMES
#telechargement du df vgsales.csv 
#telechargement du df metagames.csv

metagames = pd.read_csv("/Users/walidbenghalia//PROJET VGSALES DATASCIENTEST/metagames.csv")
vgsales = pd.read_csv("~/Desktop/vgsales.csv")
display(vgsales)
display(metagames)


#comparaison des deux dataframe 

display(vgsales.head(1))
display(metagames.head(1))

#VGSALES : Traitement
#remise à niveau du format year pour supprimer le .0

#supprimer les NAN pour convertir en int 
#vgsales.dropna(inplace = True)


#transfomation de Year en Int
vgsales['year'] = vgsales['Year'].astype(int)
vgsales

#suppression de la colonne Year mauvaise
vgsales = vgsales.drop(['Year'], axis=1)
vgsales

#metagames ; traitement

metagames

#suppression de Unnamed: 0 / Year

metagames.drop(['Unnamed: 0','Year'],axis=1,inplace=True)
display(metagames.head(1))
display(vgsales.head(1))

#stockage liste des valeurs unique Platform des deux dataFrame pour analyser les différences
list_vgsales_plt = vgsales['Platform'].unique()
display(list_vgsales_plt)

list_metagames_plt = metagames['Platform'].unique()
display(list_metagames_plt)

#Changement des valeurs unique de platfrome de metagames avec celles de vgsales

#On remplace les valeurs de metagames platform par celle de vgsales platform
dictionnaire = {'Nintendo 64':'N64','PlayStation':'PS','PlayStation 3':'PS3','Dreamcast':'DC','Xbox 360':'X360',
               'Wii':'Wii','Xbox One':'XOne','PC':'PC','PlayStation 2':'PS2','PlayStation 4':'PS4',
               'GameCube':'GC','Xbox':'XB','Wii U':'WiiU','Game Boy Advance':'GBA','3DS':'3DS','DS':'DS',
               'PlayStation Vita':'PSV','PSP':'PSP','Stadia':'SAT'}
#remplacer les valeurs de metagames platform par le dictionnaire
metagames['Platform'].replace(dictionnaire,inplace = True)

#merge de metagames et vgsales
vgsales_scores = metagames.merge(vgsales,on=['Name','year','Platform'],how='inner')
#affiche les premiéres ligne du df merger vgsales_scores
vgsales_scores

vgsales_scores.info()

#suppression des nan
vgsales_scores.dropna(inplace=True)
#dimension du df vgsales_scores
vgsales_scores.shape
(5932, 13)